## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2023, Prof. J.C. Kao, TAs T. Monsoor, R. Gore, D. Singla

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on BruinLearn as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [24]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [46]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):        
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data

P_ck = np.full(8, 1/8)
print(P_ck)


m1mean_array = np.zeros((NumClass, 97))
for i in range(NumClass):
    for j in range(97):
        m1mean_array[i][j] = np.mean(trainDataArr[i, :, j])


m1cov_matrix = np.cov(trainDataArr.reshape(-1, 97).T)
print(m1cov_matrix.shape)

modParam1 = {"pi": P_ck, "mean" : m1mean_array, "cov" : m1cov_matrix}



#====================================================#
# END YOUR CODE
#====================================================# 

#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy

print("testdata", testDataArr[0][:][0])
print(testDataArr[0,1].shape) # (97,) is the correct shape 

# I need to take element testDataArr[k][i] and iterate over all k and all i's. and then pass this through my function for probability - to see if I get the same k 

k_values = np.zeros(NumClass)
correct = 0
total = 0

for k in range(NumClass):
    for i in range(91): 
        x_vect = testDataArr[k][i]
        k_values[k] = np.log(modParam1['pi'][k]) + (modParam1["mean"][k, :]).T @ np.linalg.inv(modParam1["cov"]) @ x_vect - (1/2)*(modParam1["mean"][k, :]).T @ np.linalg.inv(modParam1["cov"]) @ (modParam1["mean"][k, :])
        k_max = np.argmax(k_values)
        total += 1
        if (k_max == k):
            correct += 1

accuracy = correct / total

print("total is", total)
print("accuracy is", accuracy)


#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 


[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
(97, 97)
testdata [ 3.  2.  1.  1.  3.  1.  1.  5.  0.  0.  0.  0.  0.  2.  1.  2.  1.  2.
  1.  2.  3.  1. 13.  0.  0.  3.  2.  0.  1.  3.  0.  0.  1.  5.  3.  0.
  0.  0.  6.  1.  0.  1.  0.  2.  1.  7.  0.  1.  0.  1.  0.  4.  1.  0.
  0.  0.  3.  2.  0.  2.  3.  1.  2.  0.  0.  0.  4.  0.  3.  0.  1.  0.
  0.  0.  0.  2.  2.  0.  1.  0.  2.  1.  4.  6.  0.  7.  2.  0.  1.  1.
  2.  2.  0.  0. 12.  0.  6.]
(97,)
total is 728
accuracy is 0.33379120879120877


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
33%


### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [49]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#

m2_cov_matrix = np.zeros((8, 97, 97))

for k in range(NumClass):
        trainDataArr_Tr = trainDataArr[k].T
        m2_cov_matrix[k] = np.cov(trainDataArr_Tr, bias = True)

print(f"Class specific covariance matrix is {m2_cov_matrix}")

modParam2 = {"pi" : P_ck, "mean" : m1mean_array, "cov" : m2_cov_matrix}
# check shapes are correct - they are 
print(np.shape(modParam2["pi"]))
print(np.shape(modParam2["mean"]))
print(np.shape(modParam2["cov"]))


#====================================================#
# END YOUR CODE
#====================================================# 


(91, 97)


ValueError: could not broadcast input array from shape (91,91) into shape (97,97)

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  


### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [ ]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 

#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  


### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [ ]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#

#====================================================#
# END YOUR CODE
#====================================================# 

#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
